In [7]:
from datetime import datetime, timezone, timedelta
import pandas as pd
from io import StringIO
import geohash2
import import_ipynb
from evaluation import to_timestamp, request_influxdb, query_tangaras


# Tangara Stations

In [8]:
# Start Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
#start_timestamp = to_timestamp('2023-03-17T00:00:00-05:00')
start_timestamp = to_timestamp('2023-04-03T00:00:00-05:00')
# End Date Time ISO 8601 Format, TZ='America/Bogota' -05:00
#end_timestamp = to_timestamp('2023-03-31T00:00:00-05:00')
end_timestamp = to_timestamp('2023-04-04T00:00:00-05:00')

print(f'Since: {start_timestamp} Until: {end_timestamp}')


Since: 1680498000000 Until: 1680584400000


In [9]:
# Query Tangaras
query = query_tangaras(start_timestamp, end_timestamp)
print(query)

SELECT DISTINCT(geo) AS "geohash" FROM "fixed_stations_01" WHERE ("geo3" = 'd29') AND time >= 1680498000000ms AND time <= 1680584400000ms GROUP BY "name";


In [10]:
# Get Data Frame Tangaras
def df_tangaras(start_timestamp, end_timestamp):
    # Query Tangaras
    query = query_tangaras(start_timestamp, end_timestamp)
    # InfluxDB API REST Request
    influxdb_request = request_influxdb(query)
    #print(influxdb_request)
    #print(influxdb_request.text)

    # Data Frame Tangaras
    df_tangaras = pd.read_csv(StringIO(influxdb_request.text), sep=",")

    # Remove/Add Columns
    df_tangaras = df_tangaras[['tags', 'geohash']]
    df_tangaras['MAC'] = df_tangaras['tags'].apply(lambda x: x.split('=')[1])
    df_tangaras['GEOLOCATION'] = df_tangaras['geohash'].apply(lambda x: " ".join(str(value) for value in list(geohash2.decode_exactly(x)[0:2])))
    df_tangaras['LATITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[0])
    df_tangaras['LONGITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[1])
    df_tangaras['tags'] = df_tangaras['tags'].apply(lambda x: f"TANGARA_{x[-4:]}")
    df_tangaras.rename(columns={'tags': 'ID', 'geohash': 'GEOHASH'}, inplace=True)
    
    # Date time when query is executed
    #tz = timezone(timedelta(hours=-5))
    #df_tangaras['DATETIME'] = datetime.now(tz=tz)

    # Set Index
    df_tangaras.set_index('ID', inplace=True)
    
    return df_tangaras


In [11]:
# Data Frame Tangaras
tangaras = df_tangaras(start_timestamp, end_timestamp)
tangaras.drop_duplicates(subset=['MAC'], inplace=True)
tangaras.head()

,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE
ID,,,,,
TANGARA_DD66,d29dbmw,D29ESP32DE0DD66,3.3267974853515625 -76.62071228027344,3.3267974853515625,-76.62071228027344
TANGARA_E752,d29dbmw,D29ESP32DE0E752,3.3267974853515625 -76.62071228027344,3.3267974853515625,-76.62071228027344
TANGARA_260A,d29dbmw,D29ESP32DE1260A,3.3267974853515625 -76.62071228027344,3.3267974853515625,-76.62071228027344
TANGARA_4B1A,d29dbmw,D29ESP32DE94B1A,3.3267974853515625 -76.62071228027344,3.3267974853515625,-76.62071228027344
TANGARA_65F2,d29dbmw,D29ESP32DE965F2,3.3267974853515625 -76.62071228027344,3.3267974853515625,-76.62071228027344


In [12]:
# Save Tangaras into CSV file
tangaras.to_csv('../data/tangaras.csv')